<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/ac-willeke/mapper-soilCondition"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" style="filter: invert(100%)"/> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://drive.google.com/drive/folders/1mEQBfa-tVViVWFt27XzUP4Wr19u1iuZm"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Mapping soil condition | Extract predictor variables using Google Earth Engine  

**Author:** Jenny Hanssen, Willeke A'Campo, Zander Venter

**Description:** Script to extract biological predictor variables for the wetland polygons using a Canopy Heigt Model from Norways national height model, Sentinel-1 and Sentinel-2 imagery. 



## Connect to Earth Engine 

In [ ]:
# The earth engine api (ee) is standard in Google Colab
import ee
import subprocess
import pandas as pd

try:
    import geemap
    print("The packages are installed and imported.")
except ImportError:
    print('The geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [ ]:
try:
    ee.Initialize() # Try to initialize earth engine
    print("Earth Engine is authenticated.")
except ee.EEException:
    ee.Authenticate() # Authenticate earth engine if initialization fails
    ee.Initialize() # init again 

In [ ]:
# import local module 
#from gee_functions import *

# define functions if local module cannot be imported
def projectFeature(feature: ee.Feature, crs_dst: str, scale: float) -> ee.Feature:
    reprojected_feature=feature.transform(crs_dst, scale)
    return reprojected_feature

def resample(image, method, projection, maxPixels):
    resampled_image = image.reduceResolution(
            # Force the next reprojection to aggregate instead of resampling.
            reducer=method,
            maxPixels=maxPixels
        ).reproject(crs=projection) #Request the data at the scale and projection of the defined proejction image.
    
    return resampled_image

def print_image_metadata(image_list):
    for image in image_list:
    # Get the image information of the image
        object_type = image.getInfo()['type']
        bandNames = image.bandNames().getInfo()
        scale = image.projection().nominalScale().getInfo()
    # Print the scale of the image
        print(f'Object Type: {object_type}')
        print(f'Image Bands: {bandNames}')
        print(f'\tscale: {scale} m/px')

        for i, band in enumerate(bandNames):
        # select and get info of the current band. 
            band_x = image.select(band)
            crs = band_x.projection().crs().getInfo()
            print(f'\tcrs: {crs}')
            print(f'\tBand {i+1}: {band}\n')

## Set project standards

In [ ]:
crs_analysis = ee.Projection('EPSG:25833') # crs used for calculations within this project
crs_mapping = ee.Projection('EPSG:3857')# crs used for displaying maps
crs_wgs84 = ee.Projection('EPSG:4326')

## Import Vector Data

In [ ]:
# Import mires fc
response_var = ee.FeatureCollection("projects/gee-base-nina/assets/mapper-soilCondition/vector/response_var")

**Reproject and Check Metadata**

In [ ]:
# Reproject Data for Mapping in GEE into standard gee. system ('EPSG:3857')
crs_dst = crs_analysis
scale = 1

response_var = response_var.map(lambda f: projectFeature(f, crs_dst, scale))

# print metadata
type = response_var.getInfo()['type']
id = response_var.getInfo()['id']
crs = response_var.first().getInfo()['geometry']['crs']['properties']['name']
print(f'{type}: "{id}" \tCRS: {crs}')

## Import Raster Data
**Terrain and Elevation Data** 


In [ ]:
## Kartverket elevation 10m - Fenoscandinavia
dtm_10m = ee.Image("users/rangelandee/NINA/Raster/Fenoscandia_DTM_10m").rename('elevation')

## Høydedata 1m DTM and DSM
dtm_col = ee.ImageCollection("users/vegar/dtm1/dtmcoll")
dsm_col = ee.ImageCollection("users/vegar/dom1/domcoll")

**Mosaic Høydedata's 1m-resolution DTM and DSM ImageCollections**

Mosaic uses the mean pixel value in case pixels overlap. 

In [ ]:
# Ensure that the correct projection parameters are used for mosaicing
# original projection of dtm/dsm 
dtm_1m_crs= dtm_col.first().projection()
dtm_1m = ee.Image(dtm_col.mean()).rename('elevation').setDefaultProjection(dtm_1m_crs)
dsm_1m = ee.Image(dsm_col.mean()).rename('elevation').setDefaultProjection(dtm_1m_crs)

**Sentinel 2 Data**

In [ ]:
# Define Sentinel 2 collection
s2Col = 'S2_SR' # or 'S2' for TOA

# Define the percentage of cloud cover below which you want to include
sceneCloudThreshold = 60

# Define the pixel cloud mask probability threshold
  # pixels above this cloud probability will be masked
cloudMaskProbability = 40

# Define time period - year of 2018 to match LUCAS sampling date
startDate = '2020-05-01'
endDate = '2020-10-01'

# Define S2 band common names
S2_BANDS = ['QA60', 'B1','B2','B3','B4', 'B5', 'B6', 'B7','B8','B11','B12']; # Sentinel bands
S2_NAMES = ['QA60','cb', 'blue', 'green', 'red', 'R1', 'R2', 'R3','nir','swir1', 'swir2']; # Common names

## Data Analysis

### Calculate Canopy Height Module  

In [ ]:
# Get canopy height model (CHM) as proxy for vegetation height
chm_1m = dsm_1m.subtract(dtm_1m).rename('CHM'); # CHM = DSM - DTM
chm_1m = chm_1m.setDefaultProjection(dtm_1m_crs)
chm_10m = ee.Image(resample(
    image=chm_1m, 
    method=ee.Reducer.mean(), 
    projection=dtm_10m.projection(),
    maxPixels=65536))
chm_10m = chm_10m.setDefaultProjection(dtm_10m.projection())

# check metadata (crs and scale)
# mosaicing sets the scale to the standard "GEE" resolution. 
image_list = [dtm_10m, dtm_1m, dsm_1m, chm_1m, chm_10m]
print_image_metadata(image_list)


### Extract Sentinel-2 variables 

**Sentinel 2 processing functions**

In [ ]:
# Function to add spectral indices to Sentinel images
def addIndices(image):
  ndbi = image.expression(
    '(SWIR - NIR) / (SWIR + NIR)', {
      'SWIR': image.select('swir1'),
      'NIR': image.select('nir'),
    }).rename('ndbi')
  # Add vegetation indices
  ndvi = image.normalizedDifference(['nir', 'red']).rename('ndvi')
  nbr = image.normalizedDifference(['nir', 'swir2']).rename("nbr")
  ndsi = image.normalizedDifference(['green', 'swir1']).rename("ndsi")
  return image.addBands(ndvi).addBands(ndbi).addBands(nbr).addBands(ndsi)


In [ ]:
# Function to add spectral indices to Sentinel images
def addIndices(image):
    ndbi = image.expression(
        '(SWIR - NIR) / (SWIR + NIR)', {
            'SWIR': image.select('swir1'),
            'NIR': image.select('nir'),
        }).rename('ndbi')
  
    # Add vegetation indices
    ndvi = image.normalizedDifference(['nir', 'red']).rename('ndvi')
    nbr = image.normalizedDifference(['nir', 'swir2']).rename('nbr')
    ndsi = image.normalizedDifference(['green', 'swir1']).rename('ndsi')
  
    return image.addBands(ndvi).addBands(ndbi).addBands(nbr).addBands(ndsi)


def uniqueValues(collection, field):
    values = ee.Dictionary(collection.reduceColumns(
        ee.Reducer.frequencyHistogram(),
        [field]
    ).get('histogram')).keys()
    return values


def dailyMosaics(imgs):
    def process_image(img):
        d = ee.Date(img.get('system:time_start'))
        day = d.get('day')
        m = d.get('month')
        y = d.get('year')
        simpleDate = ee.Date.fromYMD(y, m, day)
        return img.set('simpleTime', simpleDate.millis())

    imgs = imgs.map(process_image)

    days = uniqueValues(imgs, 'simpleTime')

    def mosaic_images(d):
        d = ee.Number.parse(d)
        d = ee.Date(d)
        t = imgs.filterDate(d, d.advance(1, 'day'))
        f = ee.Image(t.first())
        t = t.mosaic()
        t = t.set('system:time_start', d.millis())
        t = t.copyProperties(f)
        return t

    imgs = days.map(mosaic_images)
    imgs = ee.ImageCollection.fromImages(imgs)

    return imgs


def getS2_SR_CLOUD_PROBABILITY(aoi, startDate, endDate):
    primary = (
        ee.ImageCollection("COPERNICUS/S2_SR")
        .filterBounds(aoi)
        .filterDate(startDate, endDate)
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', sceneCloudThreshold)
        .select(S2_BANDS, S2_NAMES)
        .map(addIndices)
    )

    secondary = (
        ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
        .filterBounds(aoi)
        .filterDate(startDate, endDate)
    )

    innerJoined = ee.Join.inner().apply({
        'primary': primary,
        'secondary': secondary,
        'condition': ee.Filter.equals({
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    })

    def mergeImageBands(joinResult):
        return ee.Image(joinResult.get('primary')).addBands(joinResult.get('secondary'))

    newCollection = innerJoined.map(mergeImageBands)

    return ee.ImageCollection(newCollection).map(maskClouds(cloudMaskProbability)).sort('system:time_start')


def maskClouds(cloudProbabilityThreshold):
    def applyMask(img):
        cloudMask = img.select('probability').lt(cloudProbabilityThreshold)
        return img.updateMask(cloudMask)

    return applyMask

def getSentStack(aoi, startDate, endDate):
    s2Combo = getS2_SR_CLOUD_PROBABILITY(aoi, startDate, endDate)
  
    s2Cleaned = s2Combo
  
    s2Cleaned = dailyMosaics(s2Cleaned)
  
    s2Median = s2Cleaned.select(['blue', 'green', 'red', 'R1', 'R2', 'R3', 'nir', 'swir1', 'swir2']) \
        .reduce('median', 4)
  
    s2Percs = s2Cleaned.select(['ndvi', 'ndsi']) \
        .reduce(ee.Reducer.percentile([5, 25, 50, 75, 95]), 4)
  
    stDev = s2Cleaned.select(['nbr']) \
        .reduce(ee.Reducer.stdDev(), 4)
    
    ndviSummer = s2Cleaned.select('ndvi').filter(ee.Filter.calendarRange(6, 8, 'month')).median().rename('ndvi_summer')
    ndviWinter = s2Cleaned.select('ndvi').filter(ee.Filter.calendarRange(12, 2, 'month')).median().rename('ndvi_winter')
    ndviSpring = s2Cleaned.select('ndvi').filter(ee.Filter.calendarRange(9, 11, 'month')).median().rename('ndvi_spring')
    ndviFall = s2Cleaned.select('ndvi').filter(ee.Filter.calendarRange(3, 5, 'month')).median().rename('ndvi_fall')
  
    ndviFocal = s2Cleaned.select('ndvi').reduce('median', 4) \
        .reduceNeighborhood(ee.Reducer.stdDev(), ee.Kernel.square(3, 'pixels')).rename('ndvi_texture_sd')
  
    s2Stack = s2Median.addBands(s2Percs) \
        .addBands(stDev) \
        .addBands(ndviSummer) \
        .addBands(ndviWinter) \
        .addBands(ndviSpring) \
        .addBands(ndviFall) \
        .addBands(ndviFocal)
    
    # Multiply by 1000 and convert to integer to reduce file sizes on export
    s2Stack = s2Stack.multiply(1000).round().int()

    return s2Stack



In [ ]:
HIER GEBLEVEN

In [ ]:
wetlands = ee.FeatureCollection('users/zandersamuel/NINA/Vector/Norway_myrselskap_wetlands')
aoi = wetlands # response_var not working?

# Get the Sentinel stack
s2Stack = getSentStack(aoi, startDate, endDate)

# Print the Sentinel stack
print(s2Stack)


# Extract the Climatic Variables 

In [ ]:
# create raster stack for export
exportStack = chm_1m. \
  addBands(dtm_1m) 


In [ ]:
exportStack.getInfo()

In [ ]:
exportTable = exportStack.reduceRegions({
  'collection': response_var,
  'reducer': ee.Reducer.mean(),
  'scale': 10,
  'tileScale' : 4
})
print(exportTable.limit(5), 'export table')

Display the output data 

In [ ]:
import pandas as pd
df = pd.DataFrame(miresWithTerrainVariables.limit(10).getInfo()['features'])

In [ ]:
props_df = pd.json_normalize(df['properties'])
new_df = pd.concat([df,props_df], axis=1)
new_df.drop('properties',axis=1,inplace=True)
new_df.drop('geometry',axis=1,inplace=True)
display(new_df)

## Export the output data 

In [ ]:
# Store export options in dictionary
output_folder = 'GEE_output'
export_options = {
  'collection': miresWithTerrainVariables,
  'description': 'mires_terrain',
  'fileFormat': 'CSV',
  'folder': output_folder
  }

In [ ]:
import time
# Export table to Google Drive
task = ee.batch.Export.table.toDrive(**export_options)
task.start()

# Get export task status
status = task.status()['state']
while status == 'READY' or status == 'RUNNING':
    time.sleep(5)
    status = task.status()['state']

# Print export location
if status == 'COMPLETED':
    print(f"Export completed. File is stored in ...Google Drive/{output_folder}")
else:
    print("Export failed.")


### Clear Memory 

In [ ]:
# Clear data from memory
del miresWithTerrainVariables